In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import time
import plotly.graph_objects as go

In [17]:
update_dashboard()

{'data': [{'x': 549    2025-03-24 08:00:02
550    2025-03-24 08:05:02
551    2025-03-24 08:10:01
552    2025-03-24 08:15:02
553    2025-03-24 08:20:01
554    2025-03-24 08:25:01
555    2025-03-24 08:30:02
556    2025-03-24 08:35:01
557    2025-03-24 08:40:02
558    2025-03-24 08:45:01
559    2025-03-24 08:50:01
560    2025-03-24 08:55:02
561    2025-03-24 09:00:02
562    2025-03-24 09:05:02
563    2025-03-24 09:10:01
564    2025-03-24 09:15:02
565    2025-03-24 09:20:01
566    2025-03-24 09:25:02
567    2025-03-24 09:30:01
568    2025-03-24 09:35:01
569    2025-03-24 09:40:01
570    2025-03-24 09:45:01
571    2025-03-24 09:50:01
572    2025-03-24 09:55:01
573    2025-03-24 10:00:01
574    2025-03-24 10:05:02
575    2025-03-24 10:10:02
576    2025-03-24 10:15:01
577    2025-03-24 10:20:01
578    2025-03-24 10:25:02
579    2025-03-24 10:30:02
580    2025-03-24 10:35:02
581    2025-03-24 10:40:01
582    2025-03-24 10:45:01
583    2025-03-24 10:50:02
584    2025-03-24 10:55:01
585    2025-

In [2]:
files = ["AEX.csv", "BEL20.csv", "CAC40.csv", "ISEQ20.csv", "OBX.csv", "PSI.csv"]
dfs = {}

traces = []

for file in files:
    df = pd.read_csv(file, sep=';', header=None, names=['timestamp','price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['price'] = df['price'].astype(str).str.replace(',', '', regex=False).replace('N/A', None).astype(float)
    df = df[df['timestamp'].dt.weekday <= 5]
    df = df[df['timestamp'].dt.time.between(pd.to_datetime("08:00").time(), pd.to_datetime("16:30").time())]
    dfs[file] = df

    traces.append({
        'x': df['timestamp'],
        'y': df['price'],
        'type': 'line',
        'name': file.replace('.csv', '')
    })

    

In [6]:

traces_relative = []

now = pd.Timestamp.now()
today = now.normalize()

# Date du rapport à afficher (hier si < 16h40, aujourd’hui si ≥ 1640)
report_day = today if now.time() >= pd.to_datetime("16:40").time() else today - pd.Timedelta(days=1)
report_filename = f"report_{report_day.date()}.csv"

# Boucle sur chaque fichier pour les graphiques
for file in files:
    try:
        df = pd.read_csv(file, sep=';', header=None, names=['timestamp', 'price'])
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['price'] = df['price'].astype(str).str.replace(',', '', regex=False).replace('N/A', None).astype(float)
        
        df = df.sort_values('timestamp')
        df['Date'] = df['timestamp'].dt.date

        # On garde seulement les jours ouvrés
        df = df[df['timestamp'].dt.weekday < 5]

        # On garde seulement les heures de cotation
        df = df[df['timestamp'].dt.time.between(pd.to_datetime("08:00").time(), pd.to_datetime("16:40").time())]

        # Affichage cumulatif depuis le premier jour
        P0 = df['price'].iloc[0]
        df['relative'] = df['price'] / P0

        name = file.replace('.csv', '')
        open_price = df['price'].iloc[0]
        close_price = df['price'].iloc[-1]
        change = close_price - open_price
        pct_change = ((close_price - open_price) / open_price) * 100
        vol = round(df['price'].std(), 2)
        print(vol)

        name = file.replace('.csv', '')
        trace = go.Scatter(
            x=df['timestamp'],
            y=df['relative'],
            mode='lines',
            name=file.replace('.csv', '')
        )
        traces.append(trace)

    except Exception as e:
        print(f"Erreur avec le fichier {file} : {e}")
        continue



26.49
124.87
265.92
56.72
43.09
116.69


In [5]:
vol

np.float64(116.69)

In [33]:
df

,timestamp,price,Date,relative
549,2025-03-24 08:00:02,6797.55,2025-03-24,1.000000
550,2025-03-24 08:05:02,6858.56,2025-03-24,1.008975
551,2025-03-24 08:10:01,6833.15,2025-03-24,1.005237
552,2025-03-24 08:15:02,6829.30,2025-03-24,1.004671
553,2025-03-24 08:20:01,6835.08,2025-03-24,1.005521
554,2025-03-24 08:25:01,6836.18,2025-03-24,1.005683
555,2025-03-24 08:30:02,6842.58,2025-03-24,1.006624
556,2025-03-24 08:35:01,6835.60,2025-03-24,1.005598
557,2025-03-24 08:40:02,6840.12,2025-03-24,1.006263
558,2025-03-24 08:45:01,6843.36,2025-03-24,1.006739


In [31]:


print(traces)
print(type(traces))

[{'x': 0      2025-03-22 10:15:41
1      2025-03-22 10:20:01
2      2025-03-22 10:25:01
3      2025-03-22 10:30:01
4      2025-03-22 10:35:01
5      2025-03-22 10:40:01
6      2025-03-22 10:45:01
7      2025-03-22 10:50:01
8      2025-03-22 10:55:01
9      2025-03-22 11:00:02
10     2025-03-22 11:05:02
11     2025-03-22 11:10:02
12     2025-03-22 11:15:01
13     2025-03-22 11:20:01
14     2025-03-22 11:25:01
15     2025-03-22 11:30:02
16     2025-03-22 11:35:02
17     2025-03-22 11:40:01
18     2025-03-22 11:45:01
19     2025-03-22 11:50:01
20     2025-03-22 11:55:01
21     2025-03-22 12:00:01
22     2025-03-22 12:05:01
23     2025-03-22 12:10:01
24     2025-03-22 12:15:02
25     2025-03-22 12:20:02
26     2025-03-22 12:25:01
27     2025-03-22 12:30:01
28     2025-03-22 12:35:01
29     2025-03-22 12:40:01
30     2025-03-22 12:45:02
31     2025-03-22 12:50:02
32     2025-03-22 12:55:01
33     2025-03-22 13:00:01
34     2025-03-22 13:05:01
35     2025-03-22 13:10:01
36     2025-03-22 13:

In [32]:
fig = go.Figure(data=traces)

fig.update_layout(
    title='Relative Indices Growth',
    xaxis=dict(
        title='Date',
        rangebreaks=[
            {'pattern': 'day of week', 'bounds': [5, 1]},  # Week-ends
            {'pattern': 'hour', 'bounds': ['16:40', '08:00']}  # Marché fermé
        ]
    ),
    yaxis=dict(
        title='Relative variation (%)',
        tickformat='.2f',
        ticksuffix='%'
    ),
    height=500,
    width=1000
)

fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [{'x': 0      2025-03-22 10:15:41
1      2025-03-22 10:20:01
2      2025-03-22 10:25:01
3      2025-03-22 10:30:01
4      2025-03-22 10:35:01
5      2025-03-22 10:40:01
6      2025-03-22 10:45:01
7      2025-03-22 10:50:01
8      2025-03-22 10:55:01
9      2025-03-22 11:00:02
10     2025-03-22 11:05:02
11     2025-03-22 11:10:02
12     2025-03-22 11:15:01
13     2025-03-22 11:20:01
14     2025-03-22 11:25:01
15     2025-03-22 11:30:02
16     2025-03-22 11:35:02
17     2025-03-22 11:40:01
18     2025-03-22 11:45:01
19     2025-03-22 11:50:01
20     2025-03-22 11:55:01
21     2025-03-22 12:00:01
22     2025-03-22 12:05:01
23     2025-03-22 12:10:01
24     2025-03-22 12:15:02
25     2025-03-22 12:20:02
26     2025-03-22 12:25:01
27     2025-03-22 12:30:01
28     2025-03-22 12:35:01
29     2025-03-22 12:40:01
30     2025-03-22 12:45:02
31     2025-03-22 12:50:02
32     2025-03-22 12:5

In [6]:
pd.set_option('display.max_rows', None)

In [10]:
df

,timestamp,price
0,2025-03-22 10:15:41,6797.55
1,2025-03-22 10:20:01,6797.55
2,2025-03-22 10:25:01,6797.55
3,2025-03-22 10:30:01,6797.55
4,2025-03-22 10:35:01,6797.55
5,2025-03-22 10:40:01,6797.55
6,2025-03-22 10:45:01,6797.55
7,2025-03-22 10:50:01,6797.55
8,2025-03-22 10:55:01,6797.55
9,2025-03-22 11:00:02,6797.55


In [5]:
import pandas as pd
import plotly.graph_objects as go

# Exemple : dataframe avec une colonne 'datetime' et 'price'
df = df.set_index('datetime')

# Filtrer entre 8h et 17h
df_filtered = df.between_time('08:00', '17:00')

# Réinitialiser l’index pour créer un X "continu"
df_filtered = df_filtered.reset_index()

# Créer un nouvel axe X "compressé" (index numérique)
df_filtered['x_custom'] = range(len(df_filtered))

# Tracer le graphique
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_filtered['x_custom'],
    y=df_filtered['price'],
    mode='lines',
    name='Prix filtré'
))

# Affichage
fig.update_layout(
    title='Prix de 8h à 17h, avec compression des heures fermées',
    xaxis_title='Temps compressé',
    yaxis_title='Prix',
)
fig.show()


KeyError: "None of ['datetime'] are in the columns"

In [15]:
fig = {
        'data': traces,
        'layout': {
            'title': 'Indices européens (jours ouvrés, 08h - 16h30)',
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'Prix'}
        }
    }

In [16]:
fig

{'data': [{'x': 549   2025-03-24 08:00:02
   550   2025-03-24 08:05:02
   551   2025-03-24 08:10:01
   552   2025-03-24 08:15:02
   553   2025-03-24 08:20:01
   554   2025-03-24 08:25:01
   555   2025-03-24 08:30:02
   556   2025-03-24 08:35:01
   557   2025-03-24 08:40:02
   558   2025-03-24 08:45:01
   559   2025-03-24 08:50:01
   560   2025-03-24 08:55:02
   561   2025-03-24 09:00:02
   562   2025-03-24 09:05:02
   563   2025-03-24 09:10:01
   564   2025-03-24 09:15:02
   Name: timestamp, dtype: datetime64[ns],
   'y': 549    914.83
   550    920.71
   551    920.38
   552    921.28
   553    920.42
   554    919.77
   555    918.81
   556    919.31
   557    919.35
   558    919.24
   559    919.34
   560    918.79
   561    919.20
   562    918.85
   563    918.99
   564    918.51
   Name: price, dtype: float64,
   'type': 'line',
   'name': 'AEX'},
  {'x': 549   2025-03-24 08:00:02
   550   2025-03-24 08:05:02
   551   2025-03-24 08:10:01
   552   2025-03-24 08:15:02
   553   202

In [13]:
dfs

{'AEX.csv':               timestamp   price
 549 2025-03-24 08:00:02  914.83
 550 2025-03-24 08:05:02  920.71
 551 2025-03-24 08:10:01  920.38
 552 2025-03-24 08:15:02  921.28
 553 2025-03-24 08:20:01  920.42
 554 2025-03-24 08:25:01  919.77
 555 2025-03-24 08:30:02  918.81
 556 2025-03-24 08:35:01  919.31
 557 2025-03-24 08:40:02  919.35
 558 2025-03-24 08:45:01  919.24
 559 2025-03-24 08:50:01  919.34
 560 2025-03-24 08:55:02  918.79
 561 2025-03-24 09:00:02  919.20
 562 2025-03-24 09:05:02  918.85
 563 2025-03-24 09:10:01  918.99
 564 2025-03-24 09:15:02  918.51,
 'BEL20.csv':               timestamp    price
 549 2025-03-24 08:00:02  4479.88
 550 2025-03-24 08:05:02  4491.37
 551 2025-03-24 08:10:01  4486.65
 552 2025-03-24 08:15:02  4487.82
 553 2025-03-24 08:20:01  4483.36
 554 2025-03-24 08:25:01  4484.10
 555 2025-03-24 08:30:02  4482.22
 556 2025-03-24 08:35:01  4477.46
 557 2025-03-24 08:40:02  4481.16
 558 2025-03-24 08:45:01  4481.37
 559 2025-03-24 08:50:01  4480.73
 560 2